In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
%matplotlib inline

In [2]:
our_tekst = open('chatbot.txt', 'r', errors='ignore')

In [3]:
raw_text = our_tekst.read()

In [4]:
raw_text = raw_text.lower()

In [5]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
sentences = nltk.sent_tokenize(raw_text)
words = nltk.word_tokenize(raw_text)

In [7]:
sentences[:2]

['ď»żthe wars of the roses were a series of english civil wars for control of the throne of england fought between supporters of two rival branches of the royal house of plantagenet: the house of lancaster, associated with a red rose, and the house of york, whose symbol was a white rose.',
 'eventually, the wars eliminated the male lines of both families.']

In [8]:
words[:4]

['ď', '»', 'żthe', 'wars']

In [9]:
lemmer = nltk.stem.WordNetLemmatizer()

In [10]:
def Lemmatization(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [11]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return Lemmatization(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [12]:
POSSIBLE_GREETINGS = ('hello', 'hi', 'greetings', "what's up", 'hey', 'ciao' )
GREETINGS_RESPONSE = ('hi', 'hey', 'hi there', 'hello', 'ciao')

In [13]:
def Greeting(sentence):
    for word in sentence.split():
        if word.lower() in POSSIBLE_GREETINGS:
            return np.random.choice(GREETINGS_RESPONSE)
    

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
def Response(user_question):
    bot_response = ''
    
    tfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = tfidfVec.fit_transform(sentences)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    reg_tfidf = flat[-2]
    
    if(reg_tfidf == 0):
        bot_response = bot_response + 'I am sorry, I dont understand you'
        return bot_response
    else:
        bot_response = bot_response + sentences[idx]
        return bot_response

In [17]:
flag = True

print('ELLA: My name is ELLA. I will answer your questions about The War of The ROses. If you wish to exit, type Bye. ')

while(flag):
    print('\n')
    user_question = input()
    user_question = user_question.lower()
    
    if(user_question != 'bye'):
        if(user_question == 'thanks' or user_question=='thank you'):
            flag = False
            print('ELLA: You are welcome..')
        else:
            if(Greeting(user_question) != None):
                print('ELLA: ' + Greeting(user_question))
            else:
                sentences.append(user_question)
                words = words + nltk.word_tokenize(user_question)
                final_words = list(set(words))
                
                print('ELLA: ', end = '\n')
                print(Response(user_question))
                sentences.remove(user_question)
    else:
        flag = False
        print('ELLA: Bye! Take care..')

ELLA: My name is ELLA. I will answer your questions about The War of The ROses. If you wish to exit, type Bye. 


hello
ELLA: ciao


who is Edward IV
ELLA: 
york's eldest son, edward, earl of march, was proclaimed king edward iv.


who is York
ELLA: 
in the absence of a direct heir, there were two rival branches with claims to the throne should henry die without issue, being the beaufort family, led by edmund beaufort, 2nd duke of somerset, and the house of york, headed by richard of york, 3rd duke of york.


who won the war of the roses
ELLA: 
the white rose of the house of york

the red rose of the house of lancaster
the name "wars of the roses" refers to the heraldic badges associated with two rival branches of the same royal house, the white rose of york and the red rose of lancaster.


bye
ELLA: Bye! Take care..
